In [ ]:
import pandas as pd


In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive.flush_and_unmount()

In [ ]:
import io
#df=pd.read_csv(io.BytesIO(uploaded['Combined_IoT_dataset.csv']))
df=pd.read_csv('/content/drive/MyDrive/Combined_IoT_dataset.zip')

In [ ]:
df

,ts,date,time,FC1_Read_Input_Register,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,current_temperature,door_state,fridge_temperature,humidity,latitude,light_status,longitude,motion_status,pressure,sphone_signal,temp_condition,temperature,thermostat_status,label,type
0,1554061012,31-Mar-19,12:36:52,32450,32708,32035,32728,28.442693,closed,13.10,46.343618,4.514077,off,14.421946,0,1.035,false,high,35.773605,1,0,normal
1,1554061013,31-Mar-19,12:36:53,32450,32708,32035,32728,28.442693,closed,8.65,46.343618,4.514077,off,14.421946,0,1.035,false,high,35.773605,1,0,normal
2,1554061014,31-Mar-19,12:36:54,32450,32708,32035,32728,28.442693,closed,2.00,46.343618,4.514077,off,14.421946,0,1.035,false,low,35.773605,1,0,normal
3,1554061015,31-Mar-19,12:36:55,32450,32708,32035,32728,28.442693,closed,4.80,46.343618,4.514077,off,14.421946,0,1.035,false,low,35.773605,1,0,normal
4,1554061016,31-Mar-19,12:36:56,32450,32708,32035,32728,28.442693,closed,10.70,46.343618,4.514077,off,14.421946,0,1.035,false,high,35.773605,1,0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401114,1556525580,29-Apr-19,1:13:00,32450,32708,32035,32728,28.442693,closed,2.00,46.343618,4.514077,off,14.421946,0,1.035,false,low,35.773605,1,1,backdoor
401115,1556525583,29-Apr-19,1:13:03,32450,32708,32035,32728,28.442693,closed,12.00,46.343618,4.514077,off,14.421946,0,1.035,false,high,35.773605,1,1,backdoor
401116,1556525585,29-Apr-19,1:13:05,32450,32708,32035,32728,28.442693,closed,7.25,46.343618,4.514077,off,14.421946,0,1.035,false,high,35.773605,1,1,backdoor
401117,1556525587,29-Apr-19,1:13:07,32450,32708,32035,32728,28.442693,closed,5.00,46.343618,4.514077,off,14.421946,0,1.035,false,low,35.773605,1,1,backdoor


In [ ]:
x=df[['fridge_temperature','temp_condition']]
y=df[['label']]

In [ ]:
x

,fridge_temperature,temp_condition
0,13.10,high
1,8.65,high
2,2.00,low
3,4.80,low
4,10.70,high
...,...,...
401114,2.00,low
401115,12.00,high
401116,7.25,high
401117,5.00,low


In [ ]:
y

,label
0,0
1,0
2,0
3,0
4,0
...,...
401114,1
401115,1
401116,1
401117,1


In [ ]:
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
x['temp_condition']=LabelBinarizer().fit_transform(x['temp_condition'].astype('str'))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
x

,fridge_temperature,temp_condition
0,13.10,0
1,8.65,0
2,2.00,0
3,4.80,0
4,10.70,0
...,...,...
401114,2.00,0
401115,12.00,1
401116,7.25,1
401117,5.00,0


In [ ]:
norm=MinMaxScaler().fit(x)
x_norm=norm.transform(x)

In [ ]:
x_norm

array([[0.93076923, 0.        ],
       [0.58846154, 0.        ],
       [0.07692308, 0.        ],
       ...,
       [0.48076923, 1.        ],
       [0.30769231, 0.        ],
       [0.61153846, 1.        ]])

In [ ]:
import numpy as np
X=np.array(x_norm)
Y=np.array(y)

In [ ]:
Y

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [ ]:
import torch

In [ ]:
!pip install syft==0.2.5 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
from torch.utils.data import Dataset
class Sensor(Dataset):
  def __init__(self,images,labels,transform=None):
    self.data=images
    self.targets=labels
    self.to_torchtensor()
    self.transform = transform
  def to_torchtensor(self):
    self.data=torch.from_numpy(self.data)
    self.labels=torch.from_numpy(self.targets)
  def __len__(self):
    return len(self.data)
  def __getitem__(self, idx):
    sample=self.data[idx]
    target=self.targets[idx]
    if self.transform:
      sample = self.transform(sample)
    return sample,target

In [ ]:
import syft as sy
hook = sy.TorchHook(torch)

UndefinedProtocolTypeError: ignored

In [ ]:
sensor1 = sy.VirtualWorker(hook, id="sensor1")
sensor2 = sy.VirtualWorker(hook, id="sensor2")
sensor3 = sy.VirtualWorker(hook, id="sensor3")
sensor4 = sy.VirtualWorker(hook, id="sensor4")

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 100
        self.test_batch_size = 100
        self.epochs =50
        self.lr = 0.001
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 5
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.3, random_state=7)
x_train=x_train.astype(np.float32)
x_test=x_test.astype(np.float32)
y_train=y_train.astype(np.float32)
y_test=y_test.astype(np.float32)

In [ ]:
federated_train_loader = sy.FederatedDataLoader(
                         Sensor(x_train,y_train)
                         .federate((sensor1, sensor2,sensor3,sensor4)),batch_size=args.batch_size)

In [ ]:
federated_train_loader

AttributeError: ignored

In [ ]:
test_loader=torch.utils.data.DataLoader(Sensor(x_test,y_test), batch_size=args.batch_size)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(in_features=2, out_features=16)
    self.fc2 = nn.Linear(in_features=16, out_features=12)
    self.output = nn.Linear(in_features=12, out_features=1)
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.sigmoid(self.output(x))
    return x

In [ ]:
def train( args, model, device, train_loader, optimizer, epoch, criterion):
    train_loss=[]
    model.train()
    loss=0
    for batch_idx, (data, target) in enumerate(train_loader):
        model.send(data.location)
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss=criterion(output, target).get()
        print("Epoch:", epoch, "Training Loss: ", loss.item())
        train_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        model.get()

In [ ]:
def test( model, device, test_loader, criterion):
    test_loss=[]
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss =criterion(output, target)
            print("Epoch:", epoch, "Test Loss: ", loss.item())

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) 
criterion=torch.nn.BCELoss()
for epoch in range(1, args.epochs + 1):
    
    train(args, model, device, federated_train_loader, optimizer, epoch, criterion)
    test(model,device,test_loader,criterion)

Streaming output truncated to the last 5000 lines.
Epoch: 6 Training Loss:  0.6990724205970764
Epoch: 6 Training Loss:  0.6988314986228943
Epoch: 6 Training Loss:  0.6966654062271118
Epoch: 6 Training Loss:  0.6969828009605408
Epoch: 6 Training Loss:  0.6961759328842163
Epoch: 6 Training Loss:  0.6961121559143066
Epoch: 6 Training Loss:  0.6988161206245422
Epoch: 6 Training Loss:  0.698146641254425
Epoch: 6 Training Loss:  0.6996142864227295
Epoch: 6 Training Loss:  0.6950716376304626
Epoch: 6 Training Loss:  0.6978339552879333
Epoch: 6 Training Loss:  0.6971492171287537
Epoch: 6 Training Loss:  0.6938095092773438
Epoch: 6 Training Loss:  0.6999756693840027
Epoch: 6 Training Loss:  0.6964054107666016
Epoch: 6 Training Loss:  0.6930312514305115
Epoch: 6 Training Loss:  0.7004067897796631
Epoch: 6 Training Loss:  0.6996697187423706
Epoch: 6 Training Loss:  0.6975299119949341
Epoch: 6 Training Loss:  0.6995372772216797
Epoch: 6 Training Loss:  0.699510395526886
Epoch: 6 Training Loss:  0.